In [1]:
import pandas as pd
import numpy as np
from Constants import *
import Utils
import re
import simplejson
from glob import glob
import matplotlib.pyplot as plt
from re import findall, match, sub

In [2]:
class Const():
    #for troubleshooting, will migrate to own file
    data_dir = "../data/" #private data
    resource_dir = "../resources/" #public data (can be on github)
    
    mdasi_file = data_dir + "MDASI_72021.xlsx"
    camprt_dir = data_dir + "CAMPRT_Centroids/"
    
    organ_info_json = resource_dir + "OrganInfo.json"
    symptom_info_json = resource_dir + "symptoms.json"
    
    processed_organ_json = data_dir + "patient_organ_data.json"

In [3]:
def load_spatial_files(root = None):
    #reads in the files for tumor centroids and ROI-tumor distances
    #returns {'id': {'distances': <distfile>, 'doses': <centroid/dosefile>}}
    #currently only returns patients with both ids
    root = Const.camprt_dir if root is None else root
    
    try:
        distance_files = glob(Const.camprt_dir + '**/*distances.csv')
    except:
        distance_files = []
    try:
        dose_files = glob(Const.camprt_dir + '**/*centroid*.csv')
    except: 
        dose_files = []
        
    def file_id(file):
        return max([int(x) for x in findall("[0-9]+", file)])
    
    dose_dict = {file_id(f): f for f in dose_files}
    dist_dict = {file_id(f): f for f in distance_files}
    
    dose_ids = set(dose_dict.keys())
    dist_ids = set(dist_dict.keys())
    shared_ids = dose_ids.intersection(dist_ids)
    
    #print which patients didn't have matches
    dropped_ids = dose_ids.symmetric_difference(dist_ids)
    if(len(dropped_ids) > 0):
        print("missing doses", dose_ids - shared_ids)
        print("missing distances", dist_ids - shared_ids)
        
    file_dict = {sid: {'doses': dose_dict.get(sid), 'distances': dist_dict.get(sid)} for sid in shared_ids}
    return file_dict

load_spatial_files()

{3: {'doses': '../data/CAMPRT_Centroids/3/3_centroid.csv',
  'distances': '../data/CAMPRT_Centroids/3/3_distances.csv'},
 4: {'doses': '../data/CAMPRT_Centroids/4/4_centroid.csv',
  'distances': '../data/CAMPRT_Centroids/4/4_distances.csv'},
 10: {'doses': '../data/CAMPRT_Centroids/10/10_centroid.csv',
  'distances': '../data/CAMPRT_Centroids/10/10_distances.csv'},
 11: {'doses': '../data/CAMPRT_Centroids/11/11_centroid.csv',
  'distances': '../data/CAMPRT_Centroids/11/11_distances.csv'},
 17: {'doses': '../data/CAMPRT_Centroids/17/17_centroid.csv',
  'distances': '../data/CAMPRT_Centroids/17/17_distances.csv'},
 27: {'doses': '../data/CAMPRT_Centroids/27/27_centroid.csv',
  'distances': '../data/CAMPRT_Centroids/27/27_distances.csv'},
 29: {'doses': '../data/CAMPRT_Centroids/29/29_centroid.csv',
  'distances': '../data/CAMPRT_Centroids/29/29_distances.csv'},
 31: {'doses': '../data/CAMPRT_Centroids/31/31_centroid.csv',
  'distances': '../data/CAMPRT_Centroids/31/31_distances.csv'},
 3

In [4]:
class OrganData():
    
    additional_renames = {
        'Lt_Ant_Digastric_M': 'Musc_Digastric_LA',
        'Rt_Ant_Digastric_M': 'Musc_Digastric_RA',
    }
    
    #should map columns to the ones in the lists below
    file_header_renames = {
        'x coordinate': 'x',
        'y coordinate': 'y',
        'z coordinate': 'z',
        'ROI': 'roi',
        'Structure Volume': 'volume',
        'Min Value': 'min_dose',
        'Max Value': 'max_dose',
        'Mean Value': 'mean_dose'
    }
    
    roi_cols = ['Reference ROI','Target ROI']
    roi_dist_col = 'Eucledian Distance (mm)'
    
    centroid_roi_col = 'roi'
    volume_col = 'volume'
    mean_dose_col = 'mean_dose'
    centroid_cols = ['x','y','z']
    
    default_organs = [
        '_GTVp',
        '_GTVn',
        'Brainstem',
        'Oral_Cavity',
        'Larynx',
        'Musc_Masseter',
        'Tongue',
        'Musc_Sclmast',
        'Test'
    ]

    def __init__(
        self, 
        organ_info_json = None, 
        only_default_organs = True, 
        data_type = np.float16
    ):
        self.default_organs = only_default_organs
        self.data_type = data_type
        organ_info_json = Const.organ_info_json if organ_info_json is None else organ_info_json
        with open(Const.organ_info_json) as f:
            organ_dict = simplejson.load(f)
            
        if only_default_organs:
            defaults = set(OrganData.default_organs).intersection(set(organ_dict.keys()))
            excluded = set(OrganData.default_organs) - set(organ_dict.keys())
            if len(excluded) > 0:
                print("organs not found in resource file", excluded)
            defaults = sorted(defaults)
            self.organ_dict = {o:organ_dict[o] for o in defaults}
        else:
            self.organ_dict = organ_dict
        self.organ_list = self.get_organ_list()
        self.num_organs = len(self.organ_list)
    
    def get_organ_list(self, skip_gtv = True):
        olist = []
        for organ,odata in self.organ_dict.items():
            if skip_gtv and odata['tissue_type'] == 'gtv':
                continue
            pmods = odata['positional_modifiers']
            for pm in pmods:
                olist.append(organ+pm)
            if len(pmods) < 1:
                olist.append(organ)
        return sorted(olist)
            
    def oar_rename_dict(self):
        #gives a dict for renaming organs in the old CAMPRt cohort with the new organ names
        rename_dict = OrganData.additional_renames
        for organ,odata in self.organ_dict.items():
            for alt_name in odata['alt_names']:
                pm = odata['positional_modifiers']
                if len(pm) < 1:
                    rename_dict[alt_name] = organ
                else:
                    for modifier in pm:
                        alt_key = self.standardize_position_modifier(alt_name, modifier)
                        rename_dict[alt_key] = organ + modifier
        return rename_dict
    
    def standardize_position_modifier(self, old_basename, new_modifier):
        #just reconciling the way the positions are added between cohorts
        new_name = old_basename
        lmod = new_modifier.lower()
        if('_r' in lmod):
            new_name = 'Rt_' + new_name
        elif('_l' in lmod):
            new_name = 'Lt_' + new_name
        elif('rt_' in lmod):
            new_name = new_name + '_R'
        elif('lt_' in lmod):
            new_name = new_name + '_L'
        return new_name
    
    def reconcile_organ_names(self, organ_dist_df, columns = None):
        #basically tries to standardize organ names accross datasets
        if type(columns) != type(None):
            organ_dist_df = organ_dist_df[columns]
        #check that this works idk
        organ_dist_df.replace(to_replace = r'_*GTV.*N', value = '_GTVn', regex = True, inplace = True)
        return organ_dist_df.replace(self.oar_rename_dict())
    
    def reconcile_cohort_columns(self, organ_df):
        #placeholder
        return organ_df.rename(OrganData.file_header_renames,axis=1)
    
    
    def read_spatial_file(self, file):
        df = pd.read_csv(file)
        df = self.reconcile_organ_names(df)
        df = self.reconcile_cohort_columns(df)
        return df

    def format_patient_distances(self, pdist_file):
        #read the file with the centroid info, and format it for the data
        #currently outputs a dict of {(organ1, organ2): distance} where organ1, organ2 are sorted alphaetically
        dist_df = self.read_spatial_file(pdist_file)
        dist_df = dist_df.reindex(OrganData.roi_cols + [OrganData.roi_dist_col],axis=1)
        dist_df = dist_df.dropna()
        subdf = dist_df.loc[:, OrganData.roi_cols]
        dist_df.loc[:,'organ1'] = subdf.apply(lambda x: sorted(x)[1], axis=1)
        dist_df.loc[:,'organ2'] = subdf.apply(lambda x: sorted(x)[0], axis=1)
        dist_df = dist_df.set_index(['organ1','organ2']).sort_index(kind='mergesort') #I just sort everthing alphabetically, may bug out otherwise idk
        dist_df = dist_df.loc[:,OrganData.roi_dist_col]
        return dist_df.reset_index()
    
    def process_dose_file(self, dose_file, default_value = np.nan):
        dose_df = self.read_spatial_file(dose_file)
        
        dose_df = dose_df.set_index(OrganData.centroid_roi_col).sort_index()
        organs = sorted(dose_df.index.values)
        dose_dict = {}
        for organ in organs:
            #filter out extra organs idk
            if ('GTV' not in organ) and organ not in self.organ_list:
                continue
            entry = {}
            def getfield(col):
                try:
                    return dose_df.loc[organ, col]
                except:
                    return default_value
            entry[OrganData.volume_col] = getfield(OrganData.volume_col)
            entry[OrganData.mean_dose_col] = getfield(OrganData.mean_dose_col)
            entry['centroids'] = np.array([getfield(v) for v in OrganData.centroid_cols])
            dose_dict[organ] = entry
        return dose_dict
    
    def process_distance_file(self,file, centroid_dict, default_value = np.nan):
        #reads a file, returns a df with organ1, organ2, distance (sorted)
        dist_df = self.format_patient_distances(file)
        rois = set(centroid_dict.keys())
        oars = sorted(set(self.organ_list).intersection(rois))
        gtvs = [r for r in rois if 'GTV' in r]
        
        merged_dict = {}
        organs = set(list(oars) + gtvs)
        #we want the entrys to be all valid organs or gtvs, but
        #the distance array to be in the shape of the predefined list
        for o1 in organs: 
            oentry = np.zeros((self.num_organs,)).astype(self.data_type)
            if(o1 not in organs):
                oentry = oentry.fill(default_value)
            else:
                for pos, o2 in enumerate(self.organ_list):
                    if o1 == o2:
                        continue
                    if o2 not in oars:
                        tdist = default_value
                    else:
                        match = dist_df[(dist_df.organ1 == o1) & (dist_df.organ2 == o2) | (dist_df.organ1 == o2) & (dist_df.organ2 == o1)]
                        if match.shape[0] > 0:
                            tdist = match[OrganData.roi_dist_col].values
                            assert(len(tdist) < 2)
                            tdist = tdist[0]
                        else:
                            tdist = default_value
                    oentry[pos] = tdist
            mdict_entry = centroid_dict[o1]
            mdict_entry['distances'] = oentry
            merged_dict[o1] = mdict_entry
        return merged_dict
    
    def format_gtvs(self, mdict):
        gtvs = [(k,v) for k,v in mdict.items() if 'GTV' in k]
        if len(gtvs) < 1:
            return mdict
        oars = rename_gtvs(gtvs)
        for oname, odata in mdict.items():
            if 'GTV' in oname:
                continue
            oars[oname] = odata
        return oars
    
    def process_patient(self, dist_file,dose_file):
        dose_dict= self.process_dose_file(dose_file)
        merged_dict = self.process_distance_file(dist_file, dose_dict)
        merged_dict= self.format_gtvs(merged_dict)
        return merged_dict
    
    def is_valid_patient(self,p_entry):
        #code for cleaning up patients that are just no good
        has_gtv = False
        for oar in p_entry.keys():
            if 'GTV' in oar:
                has_gtv = True
                return has_gtv #delete this line if I want more stuff here
            else:
                has_gtv = False
        return has_gtv
                
    def process_cohort_spatial_dict(self, spatial_files):
        patients = {}
        invalid_ids = []
        for pid, entry in spatial_files.items():
            try:
                p_entry = self.process_patient(entry['distances'], entry['doses'])
                if(self.is_valid_patient(p_entry)):
                    patients[pid] = p_entry
                else:
                    invalid_ids.append(pid)
            except Exception as e:
                print('error reading patient', pid)
                print(e)
        if len(invalid_ids) > 1:
            print("invalid patients", invalid_ids)
        return {'organs': self.organ_list, 'patients': patients}
    
def rename_gtvs(gtvlist):
    #rename gtvs for a single patient
    new_dict = {}
    sorted_entries = sorted(gtvlist, key = lambda x: -x[1].get(OrganData.volume_col,0))
    currname = 'GTVp'
    node_num = 0
    for (gtvname, gtv) in sorted_entries:
        #stuff to error check nan could go here
        try:
            volume = float(gtv.get(OrganData.volume_col,np.nan))
            temp_name = currname
            if(node_num > 1):
                temp_name = temp_name + str(node_num)
            new_dict[temp_name] = gtv
            if(currname == 'GTVp'):
                currname = 'GTVn'
            node_num += 1
        except Exception as e:
            print('error reading gtv', gtvname, gtv)
            print(e)
    return new_dict

spatial_files = load_spatial_files()
od = OrganData()
# od.process_patient(spatial_files[3]['distances'],spatial_files[3]['doses'])
pdict = od.process_cohort_spatial_dict(spatial_files)
pdict

organs not found in resource file {'Test'}
error reading patient 27
'<' not supported between instances of 'float' and 'str'
error reading patient 10029
bad operand type for unary -: 'str'
invalid patients [17, 39, 101, 115, 10011, 10043, 10074, 10080, 10094, 10145, 10148, 10174, 10181, 2020]


{'organs': ['Brainstem',
  'Larynx',
  'Musc_Masseter_L',
  'Musc_Masseter_R',
  'Musc_Sclmast_L',
  'Musc_Sclmast_R',
  'Oral_Cavity',
  'Tongue'],
 'patients': {3: {'GTVp': {'volume': 13.1,
    'mean_dose': 71.9,
    'centroids': array([295.3126, 226.5868,  77.6118]),
    'distances': array([39.12, 13.67, 15.19, 74.44, -7.01, 47.72, 15.75, 21.94],
          dtype=float16)},
   'GTVn': {'volume': 3.1,
    'mean_dose': 71.27,
    'centroids': array([277.3846, 212.5056,  68.1895]),
    'distances': array([31.89  , 23.52  , 19.9   , 55.84  , 15.41  , 50.66  , -0.9766,
            1.953 ], dtype=float16)},
   'Oral_Cavity': {'volume': 89.0,
    'mean_dose': 47.07,
    'centroids': array([257.1202, 181.4786,  66.8123]),
    'distances': array([ 33.66,  15.  ,  20.28,  18.95,  22.2 ,  27.83,   0.  , -19.61],
          dtype=float16)},
   'Musc_Masseter_R': {'volume': 18.6,
    'mean_dose': 14.97,
    'centroids': array([208.6978, 191.491 ,  59.4124]),
    'distances': array([44.9 , 51.06, 7

In [5]:
Utils.np_dict_to_json(pdict,Const.processed_organ_json)
del pdict

In [6]:
def load_pdict(filepath = None):
    if filepath is None:
        filepath = Const.processed_organ_json
    with open(filepath) as f:
        pdict = simplejson.load(f)
    return pdict
pdata = load_pdict()
pdata 

{'organs': ['Brainstem',
  'Larynx',
  'Musc_Masseter_L',
  'Musc_Masseter_R',
  'Musc_Sclmast_L',
  'Musc_Sclmast_R',
  'Oral_Cavity',
  'Tongue'],
 'patients': {'3': {'GTVp': {'volume': 13.1,
    'mean_dose': 71.9,
    'centroids': [295.3126, 226.5868, 77.6118],
    'distances': [39.125,
     13.671875,
     15.1875,
     74.4375,
     -7.01171875,
     47.71875,
     15.75,
     21.9375]},
   'GTVn': {'volume': 3.1,
    'mean_dose': 71.27,
    'centroids': [277.3846, 212.5056, 68.1895],
    'distances': [31.890625,
     23.515625,
     19.90625,
     55.84375,
     15.40625,
     50.65625,
     -0.9765625,
     1.953125]},
   'Oral_Cavity': {'volume': 89.0,
    'mean_dose': 47.07,
    'centroids': [257.1202, 181.4786, 66.8123],
    'distances': [33.65625,
     15.0,
     20.28125,
     18.953125,
     22.203125,
     27.828125,
     0.0,
     -19.609375]},
   'Musc_Masseter_R': {'volume': 18.6,
    'mean_dose': 14.97,
    'centroids': [208.6978, 191.49099999999999, 59.4124],
    'di

In [7]:
def get_default_shape(pdict, key):
    default = None
    for pid, odata in pdict.items():
        for organ, organ_entry in odata.items():
            ovalues = organ_entry.get(key, None)
            if ovalues is not None:
                if(Utils.iterable(ovalues)):
                    return [np.nan for x in ovalues]
                else:
                    return np.nan
    print("error getting default")
    return None

def patient_organs_to_list(p_entry, key, organ_list, default):
    p_list = []
    for ii, organ in enumerate(organ_list):
        if organ not in p_entry.keys():
            values = default
        else:
            values = p_entry.get(organ).get(key)
        p_list.append(values)
    return p_list

def get_spatial_oar_array(sdict, key):
    #return n_patients x n_organs x n_values (if not a single thing)
    #doesn't include gtv
    patients = sdict['patients']
    organ_list = sdict['organs']
    pids = sorted(patients.keys())
    val_list = []
    default = get_default_shape(patients, key)
    for idx, pid in enumerate(pids):
        p_entry = patients.get(pid)
        p_list = patient_organs_to_list(p_entry, key, organ_list, default)
        val_list.append(p_list)
    return np.stack(val_list)

def aggregate_gtv_entry(gtv_list, key, aggfunc):
    #takes a single entry {'GTVp': {data}, 'GTVn': {data}, <GTVn2>...}
    vals = []
    for gdata in gtv_list:
        d = gdata.get(key,None)
        if(d is not None):
            vals.append(d)
    if len(vals) < 1:
        return None
    vals = np.stack(vals).astype('float')
    if np.isnan(vals).all():
        return None
    return np.apply_along_axis(aggfunc,0,vals)
    
def get_spatial_gtv_array(sdict, key, agg = None, filter_string = 'GTV'):
    patients = sdict['patients']
    organ_list = sdict['organs']
    default = get_default_shape(patients, key)
    #since we have an arbiratry # of gtvs, we need an aggregator
    if agg is None:
        agg = np.nanmin
    val_list = []
    for pid, p in patients.items():
        gtvs = [v for k,v in p.items() if filter_string in k]
        if(len(gtvs) < 1):
            print(pid, p.keys())
            print()
        gtv_vals = aggregate_gtv_entry(gtvs, key, agg)
        if gtv_vals is not None:
            val_list.append(gtv_vals)
        else:
            val_list.append(default)
    return np.stack(val_list)

def notnan(a):
    return np.ma.array(a, mask = np.isnan(a))

def merged_spatial_array(sdict, key, replace_nan = True, **kwargs):
    #creates a merged array with the aggregated gtv at the ned
    oar_array = get_spatial_oar_array(sdict, key)
    gtv_array = get_spatial_gtv_array(sdict, key, **kwargs)
    gtv_array =np.expand_dims(gtv_array,axis=1)
    arr = np.append(gtv_array, oar_array, axis = 1)
    if replace_nan:
        arr = np.nan_to_num(arr)
    #should be n_patients x (n_organs + gtv) x data_dims (nothing, 3 or n_organs)
    return arr

merged_spatial_array(pdata, 'volume')

array([[3.100e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.100e+00, 2.810e+01, 1.560e+01, ..., 6.430e+01, 2.090e+02,
        5.790e+01],
       [0.000e+00, 2.220e+01, 1.010e+01, ..., 3.820e+01, 9.890e+01,
        3.520e+01],
       ...,
       [8.200e+00, 2.450e+01, 1.380e+01, ..., 5.660e+01, 1.543e+02,
        4.240e+01],
       [2.000e-01, 2.280e+01, 1.040e+01, ..., 5.140e+01, 1.288e+02,
        3.750e+01],
       [2.400e+00, 2.820e+01, 2.000e+01, ..., 5.530e+01, 1.668e+02,
        4.300e+01]])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch
class OrganAutoEncoder(nn.Module):
    
    def __init__(self,
                input_size,
                hidden_dims = [150,50,6,20,150],
                init_dropout = .7,
                embedding_dropout = .2,
                ):
        
        super(OrganAutoEncoder,self).__init__()
            
        self.input_size = input_size
        self.init_dropout = init_dropout
        self.hidden_dims = hidden_dims
        self.embedding_dropout = embedding_dropout
        
        self.hidden_layers = self.init_hidden_layers()
        self.leaky_relu = nn.LeakyReLU(.2)
        self.dropout_layer = nn.Dropout(p=self.init_dropout)
        self.flatten = nn.Flatten().cuda()
        
    def init_hidden_layers(self):
        first = nn.Linear(self.input_size, self.hidden_dims[0])
        layers = [first]
        embedding_size = min(*self.hidden_dims)
        for i in range(len(self.hidden_dims)-1):
            hidden = nn.Linear(self.hidden_dims[i], self.hidden_dims[i+1])
            layers.append(hidden)
            if self.hidden_dims[i+1] <= embedding_size:
                layers.append(nn.Dropout(p=self.embedding_dropout))
            layers.append(nn.ReLU())
        
        final = nn.Linear(self.hidden_dims[-1], self.input_size)
        final_activation = nn.LeakyReLU(.1)
        layers.append(final)
        layers.append(final_activation)
        return nn.Sequential(*layers)
        
    def forward(self,x):
        #we keep in nans in the data so I can ignore them in the loss function
        x = torch.nan_to_num(x)
        xout = self.flatten(x)
        xout = self.dropout_layer(xout)
        xout = self.hidden_layers(xout)
        return xout.reshape(x.shape)
    
class Normalizer():

    def __init__(self, x):
        self.std = 1
        self.mean = 0
        self.fit(x)

    def fit(self, x):
        self.std = np.nanstd(x, axis = 0)
        self.mean = np.nanmean(x, axis = 0)

    def transform(self, x):
        x = (x - self.mean)/(self.std + .0001)
        return x

    def fit_transform(self, x):
        self.fit(x)
        return self.transform(x)

    def unnormalize(self, x):
        return x*self.std + self.mean
    
    
def nan_mse_loss(ypred, y):
    #ignores loss in the autoencoder for missing values
    y = torch.flatten(y)
    ypred = torch.flatten(ypred)
    mask = torch.isnan(y)
    out = (ypred[~mask] - y[~mask])**2
    loss = out.mean()
    return loss
    
def np_to_torch(x):
#     x = x.reshape((x.shape[0],-1))
    x = torch.tensor(x).float()
    return x
    
def train_autoencoder(sdict, key, lr = .00001, epochs = 1000000, **kwargs):
    x = merged_spatial_array(sdict, key, replace_nan = False, **kwargs)   
    normalizer = Normalizer(x)
    x = normalizer.transform(x)
    print(x.shape)
    x = torch.tensor(x).float()
    
    autoencoder = OrganAutoEncoder(x.reshape((x.shape[0],-1)).shape[-1])
#     if torch.cuda.is_available():
#         autoencoder = nn.DataParallel(autoencoder)
#         x = x.cuda()
    optimizer = torch.optim.Adam(autoencoder.parameters(), lr = lr)
    
    losses = []
    print(nan_mse_loss(torch.zeros(x.shape),x))
    for epoch in range(epochs):
        optimizer.zero_grad()
        y_pred = autoencoder(x)
        
        loss = nan_mse_loss(y_pred, x)
        loss.backward()
        losses.append(loss.item())
        optimizer.step()
        torch.cuda.empty_cache()
    plt.plot(losses[int(.1*epochs):epochs])
    print(np.min(losses))
    return autoencoder.eval(), normalizer

train_autoencoder(pdata, 'distances')

(129, 9, 8)
tensor(0.8883)


In [9]:
test = np.array([np.nan, 1, np.nan])
[x for x in test]

[nan, 1.0, nan]

In [10]:
sorted([1,2,3,4])

[1, 2, 3, 4]

In [11]:
np.nan - 10

nan

In [12]:
[x for x in reversed([1,2,3])]

[3, 2, 1]

In [13]:
[(d,i) for d,i in reversed(enumerate([1,2,3]))]

TypeError: 'enumerate' object is not reversible